In [83]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

import matplotlib.font_manager as fm
font_name = fm.FontProperties(fname="C:\\Windows\\Fonts\\malgun.ttf").get_name()
plt.rc("font", family = font_name)

import matplotlib as mlp
mlp.rcParams["axes.unicode_minus"] = False

In [84]:
import urllib.request as req
# from urllib.error import HTTPError, URLError
import requests
from bs4 import BeautifulSoup

In [85]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys

from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.chrome.service import Service

import time

In [86]:
import folium
import json

In [87]:
# from glob import glob

---

# 1. 자동차등록대수

## (1) 데이터 처리

### 1) 데이터 로드

- 서울특별시_행정동별 자동차 등록대수 현황(공공데이터포털) : 2022 기준
- https://www.data.go.kr/data/15100168/fileData.do

In [88]:
cars_raw = pd.read_csv("../../Data/차,주차장/서울특별시_행정동별 자동차 등록대수 현황_20220430.csv",
           encoding="euc-kr")

In [89]:
cars_raw.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3570 entries, 0 to 3569
Data columns (total 16 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   사용본거지 시군구  3570 non-null   object 
 1   읍면동 (행정동)  3570 non-null   object 
 2   연료         3570 non-null   object 
 3   계          3506 non-null   float64
 4   관용_승용      3570 non-null   int64  
 5   관용_승합      3570 non-null   int64  
 6   관용_화물      3570 non-null   int64  
 7   관용_특수      3570 non-null   int64  
 8   자가용_승용     3570 non-null   int64  
 9   자가용승합      3570 non-null   int64  
 10  자가용화물      3570 non-null   int64  
 11  자가용특수      3570 non-null   int64  
 12  영업용_승용     3570 non-null   int64  
 13  영업용승합      3570 non-null   int64  
 14  영업용화물      3570 non-null   int64  
 15  영업용특수      3570 non-null   int64  
dtypes: float64(1), int64(12), object(3)
memory usage: 446.4+ KB


In [90]:
car=cars_raw[["사용본거지 시군구", "읍면동 (행정동)","연료"]]

In [91]:
car.rename(columns={"사용본거지 시군구":"시군구","읍면동 (행정동)":"행정동"}, inplace=True)

C:\Users\acorn\AppData\Local\Temp\ipykernel_32384\470876743.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  car.rename(columns={"사용본거지 시군구":"시군구","읍면동 (행정동)":"행정동"}, inplace=True)


In [92]:
car.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3570 entries, 0 to 3569
Data columns (total 3 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   시군구     3570 non-null   object
 1   행정동     3570 non-null   object
 2   연료      3570 non-null   object
dtypes: object(3)
memory usage: 83.8+ KB


### 2) 시군구, 행정동 분리

In [93]:
sigungu_car=[]
for i in cars_raw["사용본거지 시군구"].str.split(" ") :
    sigungu_car.append(i[1])

In [94]:
dong_car=[]
for i in cars_raw["읍면동 (행정동)"].str.split(" ") :
    dong_car.append(i[1])

In [95]:
car["시군구"]=sigungu_car
car["행정동"]=dong_car

C:\Users\acorn\AppData\Local\Temp\ipykernel_32384\1988335006.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  car["시군구"]=sigungu_car
C:\Users\acorn\AppData\Local\Temp\ipykernel_32384\1988335006.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  car["행정동"]=dong_car


### 3) 차량 용도별 합계

In [96]:
car["관용합계"]=cars_raw["관용_승용"]+cars_raw["관용_승합"]+cars_raw["관용_화물"]+cars_raw["관용_특수"]
car["자가용합계"]=cars_raw["자가용_승용"]+cars_raw["자가용승합"]+cars_raw["자가용화물"]+cars_raw["자가용특수"]
car["영업용합계"]=cars_raw["영업용_승용"]+cars_raw["영업용승합"]+cars_raw["영업용화물"]+cars_raw["영업용특수"]
car["계"]=car["관용합계"]+car["자가용합계"]+car["영업용합계"]

C:\Users\acorn\AppData\Local\Temp\ipykernel_32384\3469447443.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  car["관용합계"]=cars_raw["관용_승용"]+cars_raw["관용_승합"]+cars_raw["관용_화물"]+cars_raw["관용_특수"]
C:\Users\acorn\AppData\Local\Temp\ipykernel_32384\3469447443.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  car["자가용합계"]=cars_raw["자가용_승용"]+cars_raw["자가용승합"]+cars_raw["자가용화물"]+cars_raw["자가용특수"]
C:\Users\acorn\AppData\Local\Temp\ipykernel_32384\3469447443.py:3: SettingWithCopyWarning: 
A value is trying to be 

In [97]:
car["계"].sum()

3182255

In [98]:
car.head()

,시군구,행정동,연료,관용합계,자가용합계,영업용합계,계
0,종로구,기타,휘발유,0,1,0,1
1,종로구,기타,휘발유,0,2,0,2
2,종로구,기타,휘발유,0,1,0,1
3,종로구,기타,경유,0,1,0,1
4,종로구,기타,경유,0,1,0,1


## (2) 구별, 행정동별 자동차등록대수

### 1) 구별

In [99]:
gu_car= car.groupby("시군구")["계"].sum()
gu_car

시군구
강남구     245487
강동구     151959
강북구      75122
강서구     206522
관악구     118273
광진구      99115
구로구     146038
금천구      90501
노원구     152375
도봉구      95608
동대문구     98603
동작구     106417
마포구     121250
서대문구     89542
서초구     176494
성동구     106021
성북구     122216
송파구     249256
양천구     151342
영등포구    145490
용산구      75234
은평구     133791
종로구      50756
중구       59744
중랑구     115099
Name: 계, dtype: int64

### 2) 광진구 행정동별

In [100]:
gj_car= car[car["시군구"]=="광진구"]
gj_car.head()

,시군구,행정동,연료,관용합계,자가용합계,영업용합계,계
557,광진구,기타,휘발유,0,1,0,1
558,광진구,기타,경유,0,2,0,2
559,광진구,기타,휘발유,0,3,0,3
560,광진구,기타1,경유,0,2,0,2
561,광진구,화양동,휘발유,0,2009,197,2206


In [101]:
gj_car.loc[gj_car["행정동"].str.contains("기타"), "행정동"] = "기타"

In [102]:
gj_dong_car = gj_car.groupby("행정동")["계"].sum()
gj_dong_car.to_csv('동별차량등록대수.csv')

# 2. 주차면수

## (1) 데이터 처리

### 1) 데이터 로드

- 서울시 주차장 (동별) 통계(서울시열린광장) : 2022 기준
- https://data.seoul.go.kr/dataList/10152/S/2/datasetView.do

In [103]:
park_raw = pd.read_csv("../../Data/차,주차장/주차장(동별).csv", skiprows=3 )
park_raw.info()
park_raw.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 452 entries, 0 to 451
Data columns (total 31 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   동별(1)       452 non-null    object
 1   동별(2)       452 non-null    object
 2   동별(3)       452 non-null    object
 3   개소 (개소)     452 non-null    int64 
 4   면수 (면수)     452 non-null    int64 
 5   개소 (개소).1   452 non-null    object
 6   면수 (면수).1   452 non-null    object
 7   개소 (개소).2   452 non-null    int64 
 8   면수 (면수).2   452 non-null    int64 
 9   개소 (개소).3   452 non-null    object
 10  면수 (면수).3   452 non-null    object
 11  개소 (개소).4   452 non-null    object
 12  면수 (면수).4   452 non-null    object
 13  개소 (개소).5   452 non-null    object
 14  면수 (면수).5   452 non-null    object
 15  개소 (개소).6   452 non-null    object
 16  면수 (면수).6   452 non-null    object
 17  개소 (개소).7   452 non-null    object
 18  면수 (면수).7   452 non-null    object
 19  개소 (개소).8   452 non-null    object
 20  면수 (면수).8 

,동별(1),동별(2),동별(3),개소 (개소),면수 (면수),개소 (개소).1,면수 (면수).1,개소 (개소).2,면수 (면수).2,개소 (개소).3,...,개소 (개소).9,면수 (면수).9,개소 (개소).10,면수 (면수).10,개소 (개소).11,면수 (면수).11,개소 (개소).12,면수 (면수).12,개소 (개소).13,면수 (면수).13
0,합계,소계,소계,309726,4501875,12304,177281,297422,4324594,11235,...,788,28279,296634,4296315,115597,282426,87393,2361583,93644,1652306
1,합계,종로구,소계,8232,92062,185,5833,8047,86229,147,...,60,1206,7987,85023,2658,6748,1448,25103,3881,53172
2,합계,종로구,사직동,362,12605,17,1422,345,11183,13,...,5,115,340,11068,76,370,64,2442,200,8256
3,합계,종로구,삼청동,343,2237,16,122,327,2115,14,...,3,114,324,2001,84,183,9,68,231,1750
4,합계,종로구,부암동,596,3533,10,141,586,3392,8,...,-,-,586,3392,315,687,145,1202,126,1503


In [104]:
park_raw.columns=["합계","시군구","행정동",
                 "합계개소","합계면수","공영개소","공영면수","민영개소","민영면수",
                 "노상개소","노상면수","노상시영개소","노상시영면수","노상구영개소","노상구영면수",
                "노외개소", "노외면수","노외시영개소","노외시영면수","노외구영개소",
                 "노외구영면수","노외민영개소","노외민영면수",
                 "건축물부설개소","건축물부설면수",
                 "건축물부설일반주택개소", "건축물부설일반주택면수",
                 "건축물부설공동주택개소", "건축물부설공동주택면수",
                 "건축물부설일반건축물개소", "건축물부설일반건축물면수"]

In [105]:
del park_raw["합계"]

In [106]:
park_raw.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 452 entries, 0 to 451
Data columns (total 30 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   시군구           452 non-null    object
 1   행정동           452 non-null    object
 2   합계개소          452 non-null    int64 
 3   합계면수          452 non-null    int64 
 4   공영개소          452 non-null    object
 5   공영면수          452 non-null    object
 6   민영개소          452 non-null    int64 
 7   민영면수          452 non-null    int64 
 8   노상개소          452 non-null    object
 9   노상면수          452 non-null    object
 10  노상시영개소        452 non-null    object
 11  노상시영면수        452 non-null    object
 12  노상구영개소        452 non-null    object
 13  노상구영면수        452 non-null    object
 14  노외개소          452 non-null    object
 15  노외면수          452 non-null    object
 16  노외시영개소        452 non-null    object
 17  노외시영면수        452 non-null    object
 18  노외구영개소        452 non-null    object
 19  노외구영면수  

In [107]:
park = park_raw[["시군구","행정동","합계면수","공영면수","민영면수"]]

In [108]:
park = park[park["행정동"] != "소계"]

### (2) 구별, 행정동별 주차면수

### 1) 구별

In [109]:
gu_park = park.groupby("시군구")[["합계면수"]].sum()
# gu_park.su

gu_park.reset_index()
gu_park.to_csv('주차면수.csv')

### 2) 광진구 행정동별

In [110]:
gj_park = park[park["시군구"]== "광진구"]
gj_dong_park = gj_park.groupby("행정동")[["합계면수"]].sum()

# 3. 구별 차량대비주차면수

In [111]:
p=pd.merge(gu_car, gu_park, on="시군구")
p.head()

,계,합계면수
시군구,,
강남구,245487,413898
강동구,151959,209959
강북구,75122,96636
강서구,206522,315812
관악구,118273,153960


In [112]:
p["차량대비면수"] = p["합계면수"]/p["계"]*100

p.to_csv('차쟝대비면수.csv')

In [113]:
p.sort_values(by="차량대비면수", ascending=False)

,계,합계면수,차량대비면수
시군구,,,
중구,59744,109153,182.701192
종로구,50756,92062,181.381512
서초구,176494,310949,176.181060
강남구,245487,413898,168.602818
마포구,121250,199423,164.472577
용산구,75234,118632,157.684026
금천구,90501,139148,153.752997
강서구,206522,315812,152.919302
동대문구,98603,148103,150.201312


In [114]:
p["차량대비면수"].mean()

141.9911576615607

In [115]:
p["정규화"]=(p["차량대비면수"]-p["차량대비면수"].min())/(p["차량대비면수"].max()-p["차량대비면수"].min())

In [116]:
p.sort_values(by="정규화", ascending=False)

,계,합계면수,차량대비면수,정규화
시군구,,,,
중구,59744,109153,182.701192,1.000000
종로구,50756,92062,181.381512,0.981872
서초구,176494,310949,176.181060,0.910437
강남구,245487,413898,168.602818,0.806340
마포구,121250,199423,164.472577,0.749606
용산구,75234,118632,157.684026,0.656357
금천구,90501,139148,153.752997,0.602359
강서구,206522,315812,152.919302,0.590907
동대문구,98603,148103,150.201312,0.553572
